# High dimensional nonlinear Poisson

We consider the nonlinear Poisson PDE
$$ -\nabla\cdot (a(u)\nabla u) = f(x), \quad x\in\Omega$$
$$ u(x) = g(x), \quad x\in\partial\Omega,$$
where $a(u)=u^3-u$ and $\Omega=[-1,1]^d$. The true solution is $u(x)=\exp(-\frac{1}{d}\sum_{i=1}^dx_i)$, and the function $f(x)$ is computed using the true solution $u(x)$.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

import time

torch.set_default_dtype(torch.float64)

In [2]:
def sample_points(N_pts, d):
    """
    Generate training samples.
    """
    # interior points
    N_domain_tot = N_pts**2
    x_int = 2 * torch.rand(N_domain_tot, d, requires_grad=True) - 1
    
    # boundary
    x_bd = np.random.normal(-1,1,size=(4*N_pts,d))
    x_bd_norm = np.linalg.norm(x_bd, axis=1, ord=np.inf) 
    x_bd = x_bd / x_bd_norm.reshape((4*N_pts,1))
    x_bd = torch.from_numpy(x_bd)
    
    return x_int, x_bd

def u_true(x):
    """
    True function.
    """
    d = x.shape[1]
    return torch.exp( -x.sum(dim=1) / d )

def f(x):
    """
    Right-hand side.
    """
    d = x.shape[1]
    
    return (-3*u_true(x)**3 + 2*u_true(x)**2) / d

def g(x):
    """
    Boundary condition.
    """
    return u_true(x)

In [3]:
class RF_PDE(nn.Module):
    """
    Random Feature model.
    """
    def __init__(self, in_features, out_features, sigma=1.0):
        super(RF_PDE, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.sigma = sigma
        self.W = nn.Parameter(torch.randn(in_features, out_features) / sigma)
        self.b = nn.Parameter(torch.rand(out_features) * 2 * torch.pi)
        
        self.model = nn.Sequential(nn.Linear(out_features, 1))

        
    def forward(self, x):
        
        u = self.model(torch.cos(x @ self.W + self.b) * torch.sqrt( torch.tensor([2 / self.out_features])))
        return u
    
    
class PINN_PDE(nn.Module):
    """
    PINN model
    """
    def __init__(self, d):
        super(PINN_PDE, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(d,64),
        nn.Tanh(),
        nn.Linear(64,64),
        nn.Tanh(),
        nn.Linear(64,1))
        
    def forward(self,x):
        u = self.model(x)
        return u
    
def loss_fn(model, x_in, x_bd, RHS, g):
    """
    Compute the loss
    """
    
    # interior:
    u = model(x_in)

    u_x = torch.autograd.grad(u, x_in, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x_in, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
        
    f = RHS(x_in)
    f = f.reshape(u.shape)
    
    residual = (1-2*u) * torch.sum(u_x ** 2, dim=1) + (u-u**2)*torch.sum(u_xx, dim=1) - f
    
    # boundary
    u_bd = model(x_bd)
    residual_bd = model(x_bd) - g(x_bd).reshape(u_bd.shape)

    return torch.mean(residual**2) + torch.Tensor([1e6]) * torch.mean(residual_bd**2)

def train(model, optimizer, x_in, x_bd, RHS, g, epochs=1000):
    
    losses = []
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        loss = loss_fn(model, x_in, x_bd, RHS, g)
        loss.backward(retain_graph=True)
        optimizer.step()
        losses.append(loss.item())
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
    return losses

## d=8

In [4]:
# generate training samples
N_pts = 30
d = 8
x_int_train, x_bd_train = sample_points(N_pts,d)

# generate test samples
N_test = 100
x_test_int, x_test_bd = sample_points(N_test, d)
u_test_int = u_true(x_test_int).detach().numpy()
u_test_bd = u_true(x_test_bd).detach().numpy()

In [5]:
##################### RF
# initialize model and optimizer
N = 100
model_RF = RF_PDE(d, N, sigma = 0.1)
optimizer = optim.Adam(model_RF.parameters(), lr=1e-2, weight_decay=0.000001)

# Train RF model
start = time.time()
losses = train(model_RF, optimizer, x_int_train, x_bd_train, f, g, epochs=1500)
end = time.time()
print(f'Computational time of RF is {end-start:.2f}')

# compute predictions and test errors
RF_pred_int = model_RF(x_test_int).detach().numpy()
RF_pred_bd = model_RF(x_test_bd).detach().numpy()

err_RF_int = np.sum( (u_test_int.reshape(RF_pred_int.shape) - RF_pred_int ) ** 2 )
err_RF_bd = np.sum( (u_test_bd.reshape(RF_pred_bd.shape) - RF_pred_bd) ** 2 )
err_RF = (err_RF_int + err_RF_bd) / (np.size(RF_pred_int) + np.size(RF_pred_bd) )
print(f'Test error of RF is {err_RF:.2e}')

Epoch 0, Loss: 2345805.1651214994
Epoch 1000, Loss: 2206.953944139111
Computational time of RF is 40.85
Test error of RF is 2.14e-01


In [6]:
##################### RF
# initialize model and optimizer
N = 100
model_RF = RF_PDE(d, N, sigma = 1)
optimizer = optim.Adam(model_RF.parameters(), lr=1e-2, weight_decay=0.000001)

# Train RF model
start = time.time()
losses = train(model_RF, optimizer, x_int_train, x_bd_train, f, g, epochs=1500)
end = time.time()
print(f'Computational time of RF is {end-start:.2f}')

# compute predictions and test errors
RF_pred_int = model_RF(x_test_int).detach().numpy()
RF_pred_bd = model_RF(x_test_bd).detach().numpy()

err_RF_int = np.sum( (u_test_int.reshape(RF_pred_int.shape) - RF_pred_int ) ** 2 )
err_RF_bd = np.sum( (u_test_bd.reshape(RF_pred_bd.shape) - RF_pred_bd) ** 2 )
err_RF = (err_RF_int + err_RF_bd) / (np.size(RF_pred_int) + np.size(RF_pred_bd) )
print(f'Test error of RF is {err_RF:.2e}')

Epoch 0, Loss: 2548137.639138973
Epoch 1000, Loss: 38.58320394592804
Computational time of RF is 38.45
Test error of RF is 3.62e-02


In [10]:
##################### RF
# initialize model and optimizer
N = 100
model_RF = RF_PDE(d, N, sigma = 5)
optimizer = optim.Adam(model_RF.parameters(), lr=1e-2, weight_decay=0.000001)

# Train RF model
start = time.time()
losses = train(model_RF, optimizer, x_int_train, x_bd_train, f, g, epochs=1500)
end = time.time()
print(f'Computational time of RF is {end-start:.2f}')

# compute predictions and test errors
RF_pred_int = model_RF(x_test_int).detach().numpy()
RF_pred_bd = model_RF(x_test_bd).detach().numpy()

err_RF_int = np.sum( (u_test_int.reshape(RF_pred_int.shape) - RF_pred_int ) ** 2 )
err_RF_bd = np.sum( (u_test_bd.reshape(RF_pred_bd.shape) - RF_pred_bd) ** 2 )
err_RF = (err_RF_int + err_RF_bd) / (np.size(RF_pred_int) + np.size(RF_pred_bd) )
print(f'Test error of RF is {err_RF:.2e}')

Epoch 0, Loss: 2092267.2922855306
Epoch 1000, Loss: 283.6742809378693
Computational time of RF is 40.47
Test error of RF is 7.31e-03


In [7]:
##################### RF
# initialize model and optimizer
N = 100
model_RF = RF_PDE(d, N, sigma = 10)
optimizer = optim.Adam(model_RF.parameters(), lr=1e-2, weight_decay=0.000001)

# Train RF model
start = time.time()
losses = train(model_RF, optimizer, x_int_train, x_bd_train, f, g, epochs=1500)
end = time.time()
print(f'Computational time of RF is {end-start:.2f}')

# compute predictions and test errors
RF_pred_int = model_RF(x_test_int).detach().numpy()
RF_pred_bd = model_RF(x_test_bd).detach().numpy()

err_RF_int = np.sum( (u_test_int.reshape(RF_pred_int.shape) - RF_pred_int ) ** 2 )
err_RF_bd = np.sum( (u_test_bd.reshape(RF_pred_bd.shape) - RF_pred_bd) ** 2 )
err_RF = (err_RF_int + err_RF_bd) / (np.size(RF_pred_int) + np.size(RF_pred_bd) )
print(f'Test error of RF is {err_RF:.2e}')

Epoch 0, Loss: 2362250.5417948063
Epoch 1000, Loss: 65.87892424631718
Computational time of RF is 38.43
Test error of RF is 4.81e-04


In [9]:
##################### RF
# initialize model and optimizer
N = 100
model_RF = RF_PDE(d, N, sigma = 20)
optimizer = optim.Adam(model_RF.parameters(), lr=1e-2, weight_decay=0.000001)

# Train RF model
start = time.time()
losses = train(model_RF, optimizer, x_int_train, x_bd_train, f, g, epochs=1500)
end = time.time()
print(f'Computational time of RF is {end-start:.2f}')

# compute predictions and test errors
RF_pred_int = model_RF(x_test_int).detach().numpy()
RF_pred_bd = model_RF(x_test_bd).detach().numpy()

err_RF_int = np.sum( (u_test_int.reshape(RF_pred_int.shape) - RF_pred_int ) ** 2 )
err_RF_bd = np.sum( (u_test_bd.reshape(RF_pred_bd.shape) - RF_pred_bd) ** 2 )
err_RF = (err_RF_int + err_RF_bd) / (np.size(RF_pred_int) + np.size(RF_pred_bd) )
print(f'Test error of RF is {err_RF:.2e}')

Epoch 0, Loss: 2250048.0641486864
Epoch 1000, Loss: 48.7816773660272
Computational time of RF is 35.57
Test error of RF is 4.05e-04
